# Import VILLESFR

In [7]:
import pandas as pd 
filepath = 'VILLESFR.json'
VILLESFR = pd.read_json(filepath, orient='records', lines=True)

# NER

In [12]:
import spacy
from spacy.tokens import DocBin, Doc
modelpath = '../resources/fr_spacy_custom_spancat_edda'
spacy_model = spacy.load(modelpath)
Doc.set_extension("metadata", default={}, force=True)

In [15]:
from transformers import PreTrainedModel, PretrainedConfig, Pipeline

class CustomModel(PreTrainedModel):
    def __init__(self, config: PretrainedConfig):
        super().__init__(config)

    def forward(self, inputs, **kwargs):
        spacydoc = spacy_model(inputs)
        spacydict = dict(spacydoc._.metadata, **spacydoc.to_json())
        return spacydict
    
    def ner_predict(self, inputs):
        spacydoc = spacy_model(inputs)
        spacydict = dict(spacydoc._.metadata, **spacydoc.to_json())
        return spacydict
    
class MyPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs, maybe_arg=2):
        return inputs

    def _forward(self, model_inputs):
        model_outputs = self.model.forward(model_inputs)
        return model_outputs

    def postprocess(self, model_outputs):
        """
        transforms my dict into a list of dict :
        [{'entity': 'O',
        'score': 0.9999761581420898,
        'word': 'A',
        'start': 0,
        'end': 1},
        ... ,
        {'entity': 'B-LOC',
        'score': 0.9996446371078491,
        'word': 'Sheffield',
        'start': 74,
        'end': 83},
        {'entity': 'O',
        'score': 0.9999758005142212,
        'word': '.',
        'start': 83,
        'end': 84}]
        """
        output_list = []
        for span in model_outputs['spans']['sc']:
            token = model_outputs['text'][span['start']:span['end']]
            
            # remove leading lowercase characters, symbols and spaces
            shift = 0
            if span['label'] in ['NP-Spatial', 'Head']:
                while token[0].islower() or not token[0].isalpha() or token[0] == ' ':
                    token = token[1:]
                    shift += 1
                span_dict = {'entity': 'B-LOC',
                            'score': 1,
                            'word': token,
                            'start': span['start'] + shift,
                            'end': span['end']
                            }
                output_list.append(span_dict)
        return output_list

In [19]:
my_ner_model = CustomModel(config = PretrainedConfig())  
my_ner_pipeline = MyPipeline(model=my_ner_model,
                             villesfr = True
                             ) 
sentence = "* ALBI, (Géog.) ville de France, capitale de l'Albigeois, dans le haut Languedoc : elle est sur le Tarn. Long. 19. 49. lat. 43. 55. 44."
rmentions = my_ner_pipeline(sentence)
rmentions

[{'entity': 'B-LOC', 'score': 1, 'word': 'France', 'start': 25, 'end': 31},
 {'entity': 'B-LOC', 'score': 1, 'word': 'Albigeois', 'start': 47, 'end': 56},
 {'entity': 'B-LOC', 'score': 1, 'word': 'Tarn', 'start': 99, 'end': 103},
 {'entity': 'B-LOC', 'score': 1, 'word': 'Languedoc', 'start': 71, 'end': 80}]

# Ranker

In [12]:
from pathlib import Path
path = '/home/antoine/Documents/GitHub/T-Res/'
from t_res.geoparser import ranking

In [20]:
# --------------------------------------
# Instantiate the ranker:
myranker = ranking.Ranker(
    method="deezymatch",
    resources_path= path + "resources/",
    strvar_parameters={"overwrite_dataset": False,},
    deezy_parameters={
        # Paths and filenames of DeezyMatch models and data:
        "dm_path": str(Path( path + "resources/deezymatch/").resolve()),
        "dm_cands": "wkdtalts",
        "dm_model": "w2v_ocr",
        "dm_output": "deezymatch_on_the_fly",
        # Ranking measures:
        "ranking_metric": "faiss",
        "selection_threshold": 50,
        "num_candidates": 10,
        "verbose": False,
        # DeezyMatch training:
        "overwrite_training": False,
        "do_test": False,
    },
)

In [14]:
# Load the resources:
myranker.mentions_to_wikidata = myranker.load_resources()

*** Loading the ranker resources.


In [21]:
myranker.deezy_on_the_fly(['ROQUEMADOUR'])[0]

{'ROQUEMADOUR': {'Roquedur': {'Score': 0.40767000000000003,
   'Candidates': {'Q863558': 1.0}},
  'Corredor': {'Score': 0.249824,
   'Candidates': {'Q3694385': 1.0, 'Q6745839': 1.0}},
  'Oradour': {'Score': 0.22409999999999997,
   'Candidates': {'Q836897': 0.04477611940298507,
    'Q190974': 0.05172413793103448,
    'Q807034': 1.0}},
  'Broomhead Reservoir': {'Score': 0.209354,
   'Candidates': {'Q62595333': 0.6666666666666666}},
  'Roman emperor': {'Score': 0.17211600000000005,
   'Candidates': {'Q2277': 0.0004914349901713002,
    'Q12544': 7.203054094936253e-05}},
  'Békourou': {'Score': 0.16608000000000003, 'Candidates': {'Q55606267': 1.0}},
  'Rosemoor': {'Score': 0.16562799999999997,
   'Candidates': {'Q3442463': 0.009708737864077669,
    'Q7368490': 0.2631578947368421}},
  'Oredo': {'Score': 0.16227000000000003,
   'Candidates': {'Q7101001': 0.8823529411764706}},
  "Centre de la mémoire d'Oradour": {'Score': 0.14904200000000004,
   'Candidates': {'Q442113': 1.0}},
  'Pendour': {'

# T-RES using DeezyMatch with REL disambiguation

REL disambiguation **filtering out microtoponyms** and without adding place of publication.

In [1]:
import os
import sys
import sqlite3
from pathlib import Path
import json
from t_res.geoparser import geode_pipe,ranking,linking
path = '/home/antoine/Documents/GitHub/T-Res/'
NER_path = '../resources/fr_spacy_custom_spancat_edda'

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
# --------------------------------------
# Instantiate the ranker:
myranker = ranking.Ranker(
    method="deezymatch",
    resources_path= path + "resources/",
    strvar_parameters={"overwrite_dataset": False,},
    deezy_parameters={
        # Paths and filenames of DeezyMatch models and data:
        "dm_path": str(Path( path + "resources/deezymatch/").resolve()),
        "dm_cands": "wkdtalts",
        "dm_model": "w2v_ocr",
        "dm_output": "deezymatch_on_the_fly",
        # Ranking measures:
        "ranking_metric": "faiss",
        "selection_threshold": 50,
        "num_candidates": 10,
        "verbose": False,
        # DeezyMatch training:
        "overwrite_training": False,
        "do_test": False,
    },
)
with sqlite3.connect( path + "resources/rel_db/embeddings_database.db") as conn:
    cursor = conn.cursor()
    mylinker = linking.Linker(
        method="reldisamb",
        resources_path= path + "resources/",
        rel_params={
            "model_path": path + "resources/models/disambiguation/",
            "data_path":  path + "experiments/outputs/data/lwm/",
            "training_split": "originalsplit",
            "db_embeddings": cursor,
            "with_publication": False,
            "without_microtoponyms": True,
            "do_test": False,
            "default_publname": "",
            "default_publwqid": "",
        },
        overwrite_training=False,
    )

In [3]:
geoparser = geode_pipe.Pipeline(geodeNERpath=NER_path,
                              myranker=myranker,
                              mylinker=mylinker)

*** Loading the ranker resources.
The DeezyMatch model is already trained!
*** Load linking resources.
  > Loading mentions to wikidata mapping.
  > Loading gazetteer.
*** Linking resources loaded!

Model path: /home/antoine/Documents/GitHub/T-Res/resources/models/disambiguation/deezymatch+10+50_originalsplit+wmtops/model eval
Loading model from given path: /home/antoine/Documents/GitHub/T-Res/resources/models/disambiguation/deezymatch+10+50_originalsplit+wmtops/model
{'mode': 'eval', 'model_path': '/home/antoine/Documents/GitHub/T-Res/resources/models/disambiguation/deezymatch+10+50_originalsplit+wmtops/model', 'prerank_ctx_window': 50, 'keep_p_e_m': 4, 'keep_ctx_ent': 3, 'ctx_window': 100, 'tok_top_n': 25, 'mulrel_type': 'ment-norm', 'n_rels': 3, 'hid_dims': 100, 'emb_dims': 300, 'snd_local_ctx_window': 6, 'dropout_rate': 0.3, 'n_epochs': 1000, 'dev_f1_change_lr': 0.915, 'n_not_inc': 10, 'eval_after_n_epochs': 5, 'learning_rate': 0.0001, 'margin': 0.01, 'df': 0.5, 'n_loops': 10, 'n_c

In [4]:
resolved = geoparser.run_text("A remarkable case of rattening has just occurred in the building trade next to the Market-street of Shefrield, but also in Lancaster. Not in Nottingham though. Not in Ashton either, nor in Salop!")
    
for r in resolved:
    print(json.dumps(r, indent=2))
    print()

  0%|          | 0/1 [00:00<?, ?it/s]

{
  "mention": "Shefrield",
  "ner_score": 1,
  "pos": 100,
  "sent_idx": 0,
  "end_pos": 109,
  "tag": "B-LOC",
  "sentence": "A remarkable case of rattening has just occurred in the building trade next to the Market-street of Shefrield, but also in Lancaster.",
  "prediction": "Q42448",
  "ed_score": 0.005,
  "cross_cand_score": {
    "Q42448": 0.664,
    "Q1862179": 0.144,
    "Q5953687": 0.071,
    "Q3461338": 0.066,
    "Q7492568": 0.028,
    "Q7492567": 0.028,
    "Q665346": 0.0
  },
  "string_match_score": {
    "Sherfield": [
      0.859,
      [
        "Q2258424"
      ]
    ],
    "Shenfield": [
      0.708,
      [
        "Q3461338",
        "Q665346"
      ]
    ],
    "Sheffield": [
      0.659,
      [
        "Q6707254",
        "Q823917",
        "Q5953687",
        "Q7492778",
        "Q1421317",
        "Q7492594",
        "Q897533",
        "Q42448",
        "Q7492565",
        "Q1862179",
        "Q4834926",
        "Q17643392",
        "Q7492570",
        "Q19509

In [5]:
sentence = "* ALBI, (Géog.) ville de France, capitale de l'Albigeois, dans le haut Languedoc : elle est sur le Tarn. Long. 19. 49. lat. 43. 55. 44."
sentence.find('ALBI')
resolved = geoparser.run_sentence(sentence, HEAD='ALBI', verbose=True)
for r in resolved:
    print(r)

  0%|          | 0/1 [00:00<?, ?it/s]

mentions : [{'entity': 'B-LOC', 'score': 1, 'mention': 'ALBI', 'start': 2, 'end': 5}, {'entity': 'B-LOC', 'score': 1, 'mention': 'France', 'start': 25, 'end': 31}, {'entity': 'B-LOC', 'score': 1, 'mention': 'Albigeois', 'start': 47, 'end': 56}, {'entity': 'B-LOC', 'score': 1, 'mention': 'Tarn', 'start': 99, 'end': 103}, {'entity': 'B-LOC', 'score': 1, 'mention': 'Languedoc', 'start': 71, 'end': 80}]
rmentions : [{'mention': 'ALBI'}, {'mention': 'France'}, {'mention': 'Albigeois'}, {'mention': 'Tarn'}, {'mention': 'Languedoc'}]
wk_cands : {'Tarn': {'Tarn': {'Score': 1.0, 'Candidates': {'Q215653': 0.8135593220338984, 'Q12772': 0.9555921052631579, 'Q1538251': 0.08695652173913043}}}, 'Albigeois': {'Albigeois': {'Score': 1.0, 'Candidates': {'Q132801': 0.050890585241730284}}}, 'ALBI': {'Albi': {'Score': 1.0, 'Candidates': {'Q94744': 0.07142857142857142, 'Q132801': 0.9211195928753181, 'Q951428': 0.3793103448275862, 'Q53749': 1.0, 'Q284698': 0.16666666666666666, 'Q700379': 0.4}}, 'Albaji': {'S

# Performance

In [6]:
import WikidataObject

In [8]:
sample = VILLESFR
verbose = True

for i, row in sample.iterrows():
    resolved = geoparser.run_sentence(row['fullcontent'],
                                      HEAD = row['head'])
    
    sample['resolved'] = resolved[0]
    
    skyline = row['gold'] in resolved[0]['cross_cand_score'].keys()
    sample['skyline'] = skyline
    
    bestPred = resolved[0]['prediction']
    sample['bestPred'] = bestPred
    if verbose:
        print(f"Head: {row['head']}")
        print(f"Gold: {row['gold']}")
        print(f"Prediction: {bestPred}")
        print(f"Skyline: {skyline}")
        print(resolved[0]['cross_cand_score'])

    if bestPred[0] != 'Q':
        acc10 = False
        sample['acc10'] = acc10

    else :
        wd_pred = WikidataObject.WikidataObject(bestPred, coordinates=resolved[0]['latlon'])
        acc10 = wd_pred._distance_to(row['gold']) <= 10
        sample['acc10'] = acc10

    if verbose:
        print(f"Accuracy 10: {acc10}")
        print()

    

  0%|          | 0/1 [00:00<?, ?it/s]

Head: Afrique
Gold: Q818608
Prediction: Q613323
Skyline: False
{'Q613323': 0.275, 'Q13727': 0.23, 'Q1747689': 0.173, 'Q12341860': 0.122, 'Q2851613': 0.119, 'Q1636204': 0.082, 'Q943336': 0.0}
Accuracy 10: False



  0%|          | 0/1 [00:00<?, ?it/s]

Head: AGDE
Gold: Q191396
Prediction: Q191396
Skyline: True
{'Q191396': 0.273, 'Q4681437': 0.158, 'Q1016861': 0.155, 'Q2028453': 0.142, 'Q120824273': 0.142, 'Q150996': 0.13, 'Q227': 0.0}
Accuracy 10: True



  0%|          | 0/1 [00:00<?, ?it/s]

...

Head: BLAYE ou BLAIE
Gold: Q202562
Prediction: Q1251125
Skyline: False
{'Q1251125': 0.198, 'Q805483': 0.188, 'Q1115946': 0.184, 'Q998521': 0.161, 'Q3242156': 0.153, 'Q997314': 0.115, 'Q2927973': 0.0}
Accuracy 10: False



  0%|          | 0/1 [00:00<?, ?it/s]

Head: BLESNEAU
Gold: Q228682
Prediction: Q228682
Skyline: True
{'Q578176': 0.205, 'Q1085877': 0.133, 'Q726523': 0.133, 'Q228682': 0.114, 'Q881062': 0.002, 'Q3404671': 0.0}
Accuracy 10: True



  0%|          | 0/1 [00:00<?, ?it/s]

Head: BLOIS
Gold: Q160927
Prediction: Q160927
Skyline: True
{'Q160927': 0.44, 'Q683059': 0.134, 'Q11680243': 0.131, 'Q1033537': 0.125, 'Q2273271': 0.106, 'Q707110': 0.064, 'Q70232': 0.0}
